<a href="https://colab.research.google.com/github/krishshah3001-prog/Algorithmic-Trading---Finance-and-Analytics-Club-Winter-Project-2025-26/blob/main/Assignment%203/250998_PART2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
df=yf.download('TCS.NS',start='2023-01-01',end='2023-12-31',interval='1d')
df.reset_index(inplace=True)
df.columns = df.columns.get_level_values(0)

/tmp/ipython-input-310732328.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df=yf.download('TCS.NS',start='2023-01-01',end='2023-12-31',interval='1d')
[*********************100%***********************]  1 of 1 completed


In [6]:
df

Price,Date,Close,High,Low,Open,Volume
0,2023-01-02,2986.108887,2991.053137,2962.074988,2985.696922,709547
1,2023-01-03,3031.796631,3039.716284,2971.322712,2977.457035,1245178
2,2023-01-04,3034.817627,3046.445635,3008.769502,3027.538834,1231668
3,2023-01-05,3031.567627,3055.555704,3004.878419,3049.055017,1826057
4,2023-01-06,2940.421875,3022.823819,2929.846914,3021.404630,2488376
...,...,...,...,...,...,...
240,2023-12-22,3626.119385,3646.933491,3567.327700,3603.361313,2413058
241,2023-12-26,3599.140869,3635.601142,3594.020165,3622.183458,1285231
242,2023-12-27,3613.981201,3620.618971,3573.016725,3602.412563,1293976
243,2023-12-28,3603.266113,3639.394641,3595.869926,3626.119101,1682889


#MACD

In [12]:
def macd(df, sp, lp, p):
    df['sp_ema'] = df['Close'].ewm(span=sp, adjust=False).mean()
    df['lp_ema'] = df['Close'].ewm(span=lp, adjust=False).mean()

    df['MACD'] = df['sp_ema'] - df['lp_ema']
    df['Signal_Line'] = df['MACD'].ewm(span=p, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['Signal_Line']

    return df


df = macd(df, 12, 26, 9)


def macd_signal(df):
    df['MACD_Signal'] = 0

    df.loc[
        (df['MACD'] > df['Signal_Line']) &
        (df['MACD'].shift(1) <= df['Signal_Line'].shift(1)),
        'MACD_Signal'
    ] = 1

    df.loc[
        (df['MACD'] < df['Signal_Line']) &
        (df['MACD'].shift(1) >= df['Signal_Line'].shift(1)),
        'MACD_Signal'
    ] = -1

    return df


df = macd_signal(df)

buy_signals = (df['MACD_Signal'] == 1).sum()
sell_signals = (df['MACD_Signal'] == -1).sum()

print("MACD Strategy – TCS (2023)")
print("Total BUY signals:", buy_signals)
print("Total SELL signals:", sell_signals)


MACD Strategy – TCS (2023)
Total BUY signals: 11
Total SELL signals: 10


#BOLLINGER BANDS

In [13]:
def bollinger_strategy(df, period=20, std=2):
    mid = df['Close'].rolling(period).mean()
    upper = mid + std * df['Close'].rolling(period).std()
    lower = mid - std * df['Close'].rolling(period).std()

    df['Signal'] = 0
    df.loc[df['Close'] < lower, 'Signal'] = 1
    df.loc[df['Close'] > upper, 'Signal'] = -1

    buy = (df['Signal'] == 1).sum()
    sell = (df['Signal'] == -1).sum()

    print("Bollinger Bands – TCS (2023)")
    print("Total BUY signals :", buy)
    print("Total SELL signals:", sell)

bollinger_strategy(df)


Bollinger Bands – TCS (2023)
Total BUY signals : 6
Total SELL signals: 14


#RSI

In [14]:
def rsi_strategy(df, period=14):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    df['Signal'] = 0
    df.loc[df['RSI'] < 30, 'Signal'] = 1
    df.loc[df['RSI'] > 70, 'Signal'] = -1

    buy = (df['Signal'] == 1).sum()
    sell = (df['Signal'] == -1).sum()

    print("RSI Strategy – TCS (2023)")
    print("Total BUY signals :", buy)
    print("Total SELL signals:", sell)

rsi_strategy(df)


RSI Strategy – TCS (2023)
Total BUY signals : 21
Total SELL signals: 43


#ADX

In [15]:
def adx_strategy(df, period=14):
    high, low, close = df['High'], df['Low'], df['Close']

    tr = pd.concat([
        high - low,
        (high - close.shift()).abs(),
        (low - close.shift()).abs()
    ], axis=1).max(axis=1)

    atr = tr.ewm(span=period, adjust=False).mean()

    plus_dm = high.diff().where(high.diff() > 0, 0)
    minus_dm = (-low.diff()).where(low.diff() < 0, 0)

    plus_di = 100 * plus_dm.ewm(span=period, adjust=False).mean() / atr
    minus_di = 100 * minus_dm.ewm(span=period, adjust=False).mean() / atr

    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    df['ADX'] = dx.ewm(span=period, adjust=False).mean()

    df['Signal'] = 0
    df.loc[df['ADX'] > 25, 'Signal'] = 1
    df.loc[df['ADX'] < 20, 'Signal'] = -1

    buy = (df['Signal'] == 1).sum()
    sell = (df['Signal'] == -1).sum()

    print("ADX Strategy – TCS (2023)")
    print("Total BUY signals :", buy)
    print("Total SELL signals:", sell)

adx_strategy(df)


ADX Strategy – TCS (2023)
Total BUY signals : 202
Total SELL signals: 13


#Stochastic Oscillator

In [16]:
def stochastic_strategy(df, k_period=14):
    low_min = df['Low'].rolling(k_period).min()
    high_max = df['High'].rolling(k_period).max()

    df['%K'] = 100 * (df['Close'] - low_min) / (high_max - low_min)

    df['Signal'] = 0
    df.loc[df['%K'] < 20, 'Signal'] = 1
    df.loc[df['%K'] > 80, 'Signal'] = -1

    buy = (df['Signal'] == 1).sum()
    sell = (df['Signal'] == -1).sum()

    print("Stochastic Oscillator – TCS (2023)")
    print("Total BUY signals :", buy)
    print("Total SELL signals:", sell)

stochastic_strategy(df)


Stochastic Oscillator – TCS (2023)
Total BUY signals : 48
Total SELL signals: 72


#ATR

In [17]:
def atr_strategy(df, period=14):
    tr = pd.concat([
        df['High'] - df['Low'],
        (df['High'] - df['Close'].shift()).abs(),
        (df['Low'] - df['Close'].shift()).abs()
    ], axis=1).max(axis=1)

    df['ATR'] = tr.ewm(span=period, adjust=False).mean()
    atr_avg = df['ATR'].rolling(period).mean()

    df['Signal'] = 0
    df.loc[df['ATR'] > atr_avg, 'Signal'] = 1
    df.loc[df['ATR'] < atr_avg, 'Signal'] = -1

    buy = (df['Signal'] == 1).sum()
    sell = (df['Signal'] == -1).sum()

    print("ATR Strategy – TCS (2023)")
    print("Total BUY signals :", buy)
    print("Total SELL signals:", sell)

atr_strategy(df)


ATR Strategy – TCS (2023)
Total BUY signals : 110
Total SELL signals: 122
